In [ ]:
import os
import csv
from collections import deque

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 1

%aimport models
%aimport utils

In [ ]:
our_recs = sorted(utils.enumerate_recording_paths('../dataset/raw/*record*'))
our_recs += sorted(utils.enumerate_recording_paths('../dataset/raw/benni*/*record*'))

recs = [models.Recording(p) for p in our_recs]
recs.sort(key=lambda r: os.path.split(r.path)[-1])
rec = recs[0]

In [ ]:
def plot_time_until_arrival(sensor):
    plt.plot((sensor['event_ts'] - sensor['event_ts'][0]),
             (sensor['arrival_ts'] - sensor['event_ts']))
    plt.title(sensor)
    plt.xlabel('Event Timestamp [sec]')
    plt.ylabel('Time until arrival [sec]')
    
def plot_raw_data(sensor):
    event_time = sensor['event_ts'] - sensor['event_ts'][0]
    for label in sensor.fieldnames[2:]:
        if label != 'accuracy':
            plt.plot(event_time, sensor[label], label=label)
    plt.title(sensor)
    plt.xlabel('Event Timestamp [sec]')
    plt.ylabel('Value')
    plt.legend()
    
def sensor_statistics(sensor):
    print('{}:'.format(sensor.name))
    mean_dt = np.diff(sensor['event_ts']).mean()
    print('\tMean event dt: {} secs'.format(mean_dt))
    print('\tMean samples/sec: {}'.format(1/(mean_dt)))

In [ ]:
plt.figure(figsize=(16,8))
for idx, sensor in enumerate(rec.sensors):
    plt.subplot(len(rec.sensors), 1, idx + 1)
    plot_time_until_arrival(sensor)
plt.tight_layout()

for r in recs:
    for idx, sensor in enumerate(r.sensors):
        sensor_statistics(sensor)


In [ ]:
plt.figure(figsize=(16,8))
for idx, sensor in enumerate(rec.sensors):
    plt.subplot(len(rec.sensors), 1, idx + 1)
    plot_raw_data(sensor)
plt.tight_layout()

In [ ]:
for sensor in rec.sensors:
    sensor_statistics(sensor)

In [ ]:
def find_closest(A, target):
    # thanks to https://stackoverflow.com/a/8929827
    idx = A.searchsorted(target)
    idx = np.clip(idx, 1, len(A)-1)
    left = A[idx-1]
    right = A[idx]
    idx -= target - left < right - target
    return idx

def rotate_cross_product(vectors, quats):
    t = np.cross(2 * -quats[:, 1:], vectors)
    return vectors + quats[:, :1] * t + np.cross(-quats[:, 1:], t)

def rotateAccelerometer(recording):
    acc = recording.Accelerometer
    rot = recording.RotationVector
    
    acc_ts = acc['event_ts']
    rot_ts = rot['event_ts']
    
    if len(acc_ts) > len(rot_ts):
        idc = find_closest(acc_ts, rot_ts)
        acc_ts = acc_ts[idc]
        vecs = acc['val_x', 'val_y', 'val_z'][idc]
        quats = rot['val_x', 'val_y', 'val_z', 'val_w']
    else:
        idc = find_closest(rot_ts, acc_ts)
        rot_ts = rot_ts[idc]
        vecs = acc['val_x', 'val_y', 'val_z']
        quats = rot['val_x', 'val_y', 'val_z', 'val_w'][idc]    
        
    rot_acc_event_ts = np.vstack([acc_ts, rot_ts]).mean(axis=0)
    rot_acc_event_ts -= rot_acc_event_ts[0]

    rot_cross = rotate_cross_product(vecs, quats)
    return rot_acc_event_ts, rot_cross
    

def plot_rec_acc_vs_rotacc(rec):
    plt.figure(figsize=(16,4.5))
    plt.subplot(2, 1, 1)
    plot_raw_data(rec.Accelerometer)

    # plt.subplot(4, 1, 2)
    # plot_raw_data(rec.RotationVector)

    rot_ts, rot_cross = rotateAccelerometer(rec)
    plt.subplot(2, 1, 2)
    plt.plot(rot_ts, rot_cross)
    plt.legend(['val_x', 'val_y', 'val_z'])
    plt.title('<Sensor "RotatedAccelerometer">')

    plt.tight_layout()

In [ ]:
import matplotlib
matplotlib.rc('font', size=18)
for r in recs[-2:]:
    plot_rec_acc_vs_rotacc(r)

In [ ]:
def plot_time_relation(*recordings):
    for i, r in enumerate(recordings):
        ts_x = r.Accelerometer['event_ts']
        ts_y = i * np.ones(len(ts_x))
        plt.plot(ts_x, ts_y)
        
plot_time_relation(*recs[-2:-1])

In [ ]:
rec_ps = [p for p in utils.enumerate_recording_paths('../dataset')]
rec_ps.sort()
rec_ps = rec_ps[-2:]
for r in rec_ps:
    D = np.load(os.path.join(r, 'recording.npz'))
    M = D['merged']
    plt.figure(figsize=(16,8))
    for i in range(2, M.shape[1]):
        plt.plot(M[:, 0], M[:, i])
    

In [ ]:
def plot_rot_acc(df):
    keys = 'Rotated_Accelerometer_val_x', 'Rotated_Accelerometer_val_y', 'Rotated_Accelerometer_val_z'
    event_ts = df.as_matrix(['event_ts'])
    event_ts -= event_ts[0]
    for k in keys:
        plt.plot(event_ts, df[k], label=k)
    plt.xlabel('Event Timestamp [sec]')
    plt.ylabel('Value')
    plt.legend()

recs = sorted(utils.enumerate_recording_paths('../dataset'))
for r in recs[-2:]:
    file_p = os.path.join(r, 'merged.msg')
    df = pd.read_msgpack(file_p)
    plot_rot_acc(df)